In [ ]:
import torch

"""
演示自动微分模块，具体如何求导
"""

# 定义变量，记录出书的权重w旧
w = torch.tensor(10, requires_grad=True, dtype=torch.float32)
w

# 定义loss变量，表示损失函数
loss = 2 * w**2  # loss = 2w^2 --> 求导：4w
loss

# 计算梯度
# loss.sum().backward() #保证loss是一个标量，才能求导
loss.backward()  # 这里因为w本身就是一个标量，所以可以直接求导
w.grad

# 带入权重更新公式 w新 = w旧 - 学习率 * 梯度
lr = 0.01
w = w - lr * w.grad

print("更新后的权重为:", w) #9.6

更新后的权重为: tensor(9.6000, grad_fn=<SubBackward0>)


In [6]:
"""
演示自动微分，循环实现，计算梯度，更新参数

需求：求loss=w**2 + 20的极小值点，并打印y是最小值时w的值（梯度）
"""

w = torch.tensor(10, requires_grad=True, dtype=torch.float32)
loss = w**2 + 20  # 求导loss' = 2w

# 迭代100次
for i in range(1, 101):
    # 正向传播
    loss = w**2 + 20
    # 梯度清零，默认梯度会累加，需要清零
    if w.grad is not None:
        w.grad.zero_()
    # 反向传播
    loss.backward()
    # 梯度更新
    w.data = w.data - lr * w.grad
    # 打印梯度
    print(f'第{i}次，权重初始值:{w}, 梯度：{w.grad}, loss {loss}')

第1次，权重初始值:9.800000190734863, 梯度：20.0, loss 120.0
第2次，权重初始值:9.604000091552734, 梯度：19.600000381469727, loss 116.04000091552734
第3次，权重初始值:9.411920547485352, 梯度：19.20800018310547, loss 112.23681640625
第4次，权重初始值:9.223682403564453, 梯度：18.823841094970703, loss 108.5842514038086
第5次，权重初始值:9.03920841217041, 梯度：18.447364807128906, loss 105.0763168334961
第6次，权重初始值:8.858424186706543, 梯度：18.07841682434082, loss 101.70729064941406
第7次，权重初始值:8.681255340576172, 梯度：17.716848373413086, loss 98.4716796875
第8次，权重初始值:8.507630348205566, 梯度：17.362510681152344, loss 95.36419677734375
第9次，权重初始值:8.337477684020996, 梯度：17.015260696411133, loss 92.37977600097656
第10次，权重初始值:8.170727729797363, 梯度：16.674955368041992, loss 89.51353454589844
第11次，权重初始值:8.007312774658203, 梯度：16.341455459594727, loss 86.76078796386719
第12次，权重初始值:7.847166538238525, 梯度：16.014625549316406, loss 84.11705780029297
第13次，权重初始值:7.690223217010498, 梯度：15.69433307647705, loss 81.57801818847656
第14次，权重初始值:7.536418914794922, 梯度：15.380446434020996, lo

In [16]:
"""
将张量转化为ndarray
"""

t1 = torch.tensor([10, 20],requires_grad=True,dtype=torch.float32)
print(f't1:{t1},type:{type(t1)}')

# 尝试把上述的张量转换为numpy数组
# t2 = t1.numpy() #直接转换会报错
# print(f't1_np:{t1_np},type:{type(t1_np)}')

# 解决办法：通过detach函数，将张量从计算图中分离出来，转换为numpy数组
t2 = t1.detach()
print(f"t2:{t2},type:{type(t2)}")

# t1和t2共享同一块内存，修改t1会影响t2
t1.data[0] = 100
print(f"t1:{t1},type:{type(t1)}")
print(f"t2:{t2},type:{type(t2)}")

# 查看t1 和 t2 可以自动微分
print(f"t1:{t1}{t1.requires_grad}")  # True
print(f"t2:{t2}{t2.requires_grad}")  # False

# 转为numpy
n1 = t2.numpy()
print(f"n1:{n1},type:{type(n1)}")

# 总结：
n2 = t1.detach().numpy()
print(f"n2:{n2},type:{type(n2)}")

t1:tensor([10., 20.], requires_grad=True),type:<class 'torch.Tensor'>
t2:tensor([10., 20.]),type:<class 'torch.Tensor'>
t1:tensor([100.,  20.], requires_grad=True),type:<class 'torch.Tensor'>
t2:tensor([100.,  20.]),type:<class 'torch.Tensor'>
t1:tensor([100.,  20.], requires_grad=True)True
t2:tensor([100.,  20.])False
n1:[100.  20.],type:<class 'numpy.ndarray'>
n2:[100.  20.],type:<class 'numpy.ndarray'>


In [ ]:
"""
自动微分模块应用

输入数据：两个样本，每个样本5个特征
      x1  x2  x3  x4  x5
x = [[1., 1., 1., 1., 1.],
     [1., 1., 1., 1., 1.]]

# 权重
w = [[w11, w21, w31],
     [w12, w22, w32],
     [w13, w23, w33],
     [w14, w24, w34],
     [w15, w25, w35]]

# 偏置
b = [b1, b2, b3]

输出数据：
y = [[0., 0., 0.],
     [0., 0., 0.]]

y = xw + b
"""

# 1.定义x，表示输入数据，假设2行5列，全1矩阵
x = torch.ones(2, 5)


# 2.定义y,表示标签（真实值），假设2行3列，全0矩阵
y = torch.zeros(2, 3)

# 3.初始化权重和偏置（自动微分）
w = torch.randn(5, 3, requires_grad=True)
b = torch.randn(3, requires_grad=True)
print(w)
print(b)

# 4.前向传播，计算出预测值
z = torch.matmul(x , w) + b
print(z)

# 定义损失函数
criterion = torch.nn.MSELoss() # 均方差
loss = criterion(z, y)
print(loss)

## 5.反向传播，计算梯度
loss.backward()
print(w.grad)
print(b.grad)

## 后续更新梯度

tensor([[-0.2116,  0.1743,  1.6572],
        [-1.4358, -0.9859,  0.9192],
        [ 1.9637, -1.0080, -0.7727],
        [-0.6106, -1.2267, -0.0430],
        [-0.5982,  0.0159,  1.5393]], requires_grad=True)
tensor([-0.1323,  0.0406,  1.6879], requires_grad=True)
tensor([[-1.0249, -2.9899,  4.9880],
        [-1.0249, -2.9899,  4.9880]], grad_fn=<AddBackward0>)
tensor(11.6234, grad_fn=<MseLossBackward0>)
tensor([[-0.6832, -1.9932,  3.3254],
        [-0.6832, -1.9932,  3.3254],
        [-0.6832, -1.9932,  3.3254],
        [-0.6832, -1.9932,  3.3254],
        [-0.6832, -1.9932,  3.3254]])
tensor([-0.6832, -1.9932,  3.3254])
